# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [66]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


C:\Users\Yellow\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['permutation', 'clf']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [67]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [68]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    with open(filename, 'r+b') as file:
        data = pickle.load(file)
    target = [i] * len(data)
    return data, target

In [69]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
i = 0
while i < len(classes):
    all_data += load_pose_data(i)[0]
    all_target += load_pose_data(i)[1]
    i += 1
print('total number of data', len(all_data))

#print(all_data[0])
#print(all_data[1])
#print(all_data[len(all_data) - 1])
#print(all_target)

total number of data 222


In [78]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]
test_data = permutation[(int)(len(all_data) * 0.7):]
print(permutation)
print(n_training_data)
print(training_data)
print(test_data)


[ 55  11 149  77 168  30  52 102 191 221 171  64  62  80 143 163 211 104
 197 155 164  81 156 113 127  61 182 139 178  40 188 207  21  47  45 217
 135 130 148  88  32  91 180  18  48   4 175 161   7  51  78 133  14 142
 187 166 158 201 193  99  29  54  76 174  59 210 179  93  84 151  90  12
 185  13 215  97 208  25 144 172 110 203  28 112 118 196 189 115   8 137
  37 145  86  34 134 128  71  10 114 162 209 111   0  35 123  39 107  83
 177 218 183 173 153 132 103 219  87 205   6  79  70 101  82  92 220  44
  57  65  20 141 186 122 147 136 169 167  96   3 199 170  19  60  89  42
 206 200 176 157  26 184 125 195 120 131  43  75   9 116  95  33  16 202
 216 152  36 146 150 121  56 124   1 140 106  73  31  63  24 204 109 165
 129  41 100 160 117 181 194   2  38  72  85  67 138 192 213  98 159  53
   5  58 198 190  69  74 212  66 214  23  22 154 119 126 105  49  50  94
  15  68  27  46 108  17]
155
[ 55  11 149  77 168  30  52 102 191 221 171  64  62  80 143 163 211 104
 197 155 164  81 156 

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [71]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [72]:
clf.fit(np.array(all_data)[training_data], np.array(all_target)[training_data]) 

SVC(C=100.0, gamma=0.001)

In [91]:
clf.fit(all_data, all_target) 

SVC(C=100.0, gamma=0.001)

### predicting

In [80]:
prediction = clf.predict(all_data[-1:])

In [81]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [83]:
expected = []
predicted = []
# YOUR CODE HERE
predicted = clf.predict(np.array(all_data)[training_data])
expected = np.array(all_target)[training_data]
#evaluate(expected, predicted)

## 4. Evaluate on the test data

In [84]:
expected = []
predicted = []
# YOUR CODE HERE
predicted = clf.predict(np.array(all_data)[test_data])
expected = np.array(all_target)[test_data]
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00        11
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         4
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00         3
           8       1.00      1.00      1.00         6
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00        11

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[ 8  0  0  0  0  0  0  0  0  0  0]
 [ 0  8  0  0  0  0  0  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [92]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
with open(ROBOT_POSE_CLF, 'wb') as file:
    pickle.dump(clf, file)

Then, in the application we can load the trained classifier again.

In [93]:
with open(ROBOT_POSE_CLF, 'rb') as file:
    clf2 = pickle.load(file)
clf2.predict(all_data)

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  2,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10])